In [1]:
import math
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

# Weights & Biases (wandb) Setup

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret('wandb_api')

!pip install wandb --upgrade -q
import wandb
wandb.login(key=wandb_api)

run = wandb.init(
    project='G2Net',
    entity='jxiong21029',
    config={
        'epochs': 4,
        'use_tpu': True,
        'lr': 5e-4,
        'batch_size': 512,
        'model_name': 'tf_efficientnetv2_m',
        'wave_transform_params': {
            'sr': 2048, 'fmin': 20, 'fmax': 500, 'hop_length': 28, 'bins_per_octave': 24, 'filter_scale': 0.5, 'output_format': 'Magnitude'
        },
        'valid_split': 0.05
    }
)

config = wandb.config

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jxiong21029 (use `wandb login --relogin` to force relogin)
2021-09-16 15:10:55.501267: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-16 15:10:55.501978: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Load & Preprocess Data

## Load Data Files

In [3]:
def id_to_path(image_id: str, is_train: bool = True) -> str:
    folder = 'train' if is_train else 'test'
    return '../input/g2net-gravitational-wave-detection/{}/{}/{}/{}/{}.npy'.format(
        folder, image_id[0], image_id[1], image_id[2], image_id 
    )

In [4]:
train_df = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
train_df

,id,target
0,00000e74ad,1
1,00001f4945,0
2,0000661522,0
3,00007a006a,0
4,0000a38978,1
...,...,...
559995,ffff9a5645,1
559996,ffffab0c27,0
559997,ffffcf161a,1
559998,ffffd2c403,0


In [5]:
train_df['target'].value_counts()

0    280070
1    279930
Name: target, dtype: int64

## Preprocess: Butterworth Bandpass Filter

In [6]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs=2048, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def bandpass_filter(data, lowcut, highcut, fs=2048, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

## DataLoader & Device Setup

In [7]:
if config.use_tpu:
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
if config.model_name.startswith('lukemelas-efficientnet'):
    !pip install efficientnet_pytorch -q
    from efficientnet_pytorch import EfficientNet
elif config.model_name.startswith('transformer'):
    pass
else:
    import sys
    sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
    import timm 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

if config.use_tpu:
    import torch_xla
    import torch_xla.core.xla_model as xm

    device = xm.xla_device()
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_tensor_type('torch.FloatTensor')

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  66452      0 --:--:-- --:--:-- --:--:-- 67315
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.7.1+cpu



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



Uninstalling torch-1.7.1+cpu:
  Successfully uninstalled torch-1.7.1+cpu
Found existing installation: torchvision 0.8.2+cpu
Uninstalling torchvision-0.8.2+cpu:
  Successfully uninstalled torchvision-0.8.2+cpu
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl...

Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl...

Operation completed over 1 objects/119.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl...

Operation completed over 1 objects/2.3 MiB.                                      
Processing ./torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
Done updating TPU runtime
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.

In [8]:
!pip install -q nnAudio
from nnAudio import Spectrogram

In [9]:
class GravWaveDataset(torch.utils.data.Dataset):
    def __init__(self, idxs, targets):
        self.idxs = np.asarray(idxs)
        self.targets = np.asarray(targets)
        self.wave_transform = Spectrogram.CQT1992v2(**config.wave_transform_params)
          
    def __len__(self):
        return len(self.idxs)
    
    def _frequency_transform(self, x):
        channels = []
        for i in range(3):
            wave = bandpass_filter(x[i], 20, 500)
            wave /= np.max(wave)
            wave = torch.from_numpy(wave).float()
            channels.append(self.wave_transform(wave))
        
        return torch.cat(channels)
    
    def __getitem__(self, index):
        x = np.load(id_to_path(self.idxs[index])).astype(np.float32)
        image = self._frequency_transform(x)
        y = torch.tensor(self.targets[index], dtype=torch.float32)
        return image, y

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df['id'], train_df['target'], test_size=config.valid_split)

train_dataset = GravWaveDataset(X_train, y_train)
valid_dataset = GravWaveDataset(X_valid, y_valid)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size, num_workers=8, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=config.batch_size, num_workers=8)

CQT kernels created, time used = 0.0258 seconds
CQT kernels created, time used = 0.0235 seconds


/opt/conda/lib/python3.7/site-packages/nnAudio/utils.py:326: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn('If fmax is given, n_bins will be ignored',SyntaxWarning)


In [11]:
example_image = train_dataset[0][0]
example_image.shape

torch.Size([3, 112, 147])

# Model & Train Loop

In [12]:
class EffNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        if config.model_name.startswith('lukemelas-efficientnet'):
            self.net = EfficientNet.from_pretrained(f'efficientnet-{config.model_name[-2:]}')
            self.net._fc = nn.Linear(in_features=self.net._fc.in_features, out_features=1)
        else:
            self.net = timm.create_model(config.model_name, pretrained=True, in_chans=3)
            self.net.classifier = nn.Linear(self.net.classifier.in_features, 1)
    
    def forward(self, x):
        out = self.net(x)
        return out
    
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

    
class TransformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        num_ch = train_dataset[0][0].shape[1]
#         num_feat = config.model_specs['nhead'] * math.ceil(num_ch / config.model_specs['nhead'])
#         print(f'd_model={num_feat}')
        num_feat = config.model_specs['d_model']

        seq_len = train_dataset[0][0].shape[2]
#         print(num_feat, seq_len)
        self.pt_conv = nn.Conv1d(3 * num_ch, num_feat, 1)
        self.layer_norm = nn.LayerNorm((num_feat, seq_len))
        self.pos_encoding = PositionalEncoding(d_model=num_feat, dropout=config.model_specs['dropout'], max_len=seq_len)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=num_feat,
            nhead=config.model_specs['nhead'],
            dropout=config.model_specs['dropout'],
            activation=config.model_specs['activation']
        )
        self.net = nn.TransformerEncoder(encoder_layer, num_layers=config.model_specs['layers'])
        self.fc = nn.Linear(seq_len * num_feat, 1)
    
    def forward(self, x):
        x = torch.cat([x[:, 0], x[:, 1], x[:, 2]], dim=1)
        x = self.pt_conv(x)
        x = F.gelu(self.layer_norm(x))
        x = x.permute(2, 0, 1)
        x = self.pos_encoding(x)
        x = self.net(x)
        x = torch.flatten(x.permute(1, 0, 2), start_dim=1)
        out = self.fc(x)
        return out
        

In [13]:
def validation_epoch():
    model.eval()
    with torch.no_grad():
        tot_loss = 0
        predictions = np.zeros(len(valid_dataset))
        for i, (images, targets) in enumerate(valid_dataloader):
            images, targets = images.to(device), targets.to(device)
            outputs = model(images).squeeze(1)
            tot_loss += criterion(outputs, targets) * len(outputs)
            predictions[i*config.batch_size:(i+1)*config.batch_size] = torch.sigmoid(outputs).cpu().numpy().squeeze()
        wandb.log({'valid_loss': tot_loss / len(valid_dataset), 'auc': roc_auc_score(y_valid, predictions)})
    model.train()
        

In [14]:
if config.model_name.startswith('transformer'):
    model = TransformerModel().to(device).float()
else:
    model = EffNetModel().to(device).float()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# wandb.watch(model)

log_interval = 50
valid_interval = int(len(train_dataset) / config.batch_size / 3)
for epoch in range(config.epochs):
    running_loss = 0
    for i, (images, targets) in tqdm(enumerate(train_dataloader), total=math.ceil(len(train_dataset) / config.batch_size), smoothing=0):
        optimizer.zero_grad()
        
        images, targets = images.to(device), targets.to(device)
                
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, targets)
        loss.backward()
        if config.use_tpu:
            xm.optimizer_step(optimizer, barrier=True)
        else:
            optimizer.step()
        
        running_loss += loss.item()
        
        if i % log_interval == log_interval - 1:
            wandb.log({'loss': running_loss / log_interval})
            running_loss = 0
        if i % valid_interval == valid_interval // 2:
            validation_epoch()
            torch.save(model.state_dict(), '/kaggle/working/ckpt.pth')
torch.save(model.state_dict(), '/kaggle/working/ckpt.pth')
# wandb.save('/kaggle/working/ckpt.pth')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_m-cc09e0cd.pth" to /root/.cache/torch/checkpoints/tf_efficientnetv2_m-cc09e0cd.pth


  0%|          | 0/1040 [00:00<?, ?it/s]

E0916 15:22:27.312559862     939 backup_poller.cc:118]       run_poller: {"created":"@1631805747.312515508","description":"Shutting down timer system","file":"external/com_github_grpc_grpc/src/core/lib/iomgr/timer_generic.cc","file_line":706}


  0%|          | 0/1040 [00:00<?, ?it/s]

  0%|          | 0/1040 [00:00<?, ?it/s]

  0%|          | 0/1040 [00:00<?, ?it/s]

# Generate Predictions

In [15]:
test_df = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
test_df.head()

,id,target
0,00005bced6,0.5
1,0000806717,0.5
2,0000ef4fe1,0.5
3,00020de251,0.5
4,00024887b5,0.5


In [16]:
class GravWaveUnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, idxs):
        self.idxs = np.asarray(idxs)
        self.wave_transform = Spectrogram.CQT1992v2(**config.wave_transform_params)
          
    def __len__(self):
        return len(self.idxs)
    
    def _frequency_transform(self, x):
        channels = []
        for i in range(3):
            wave = bandpass_filter(x[i], 20, 500)
            wave /= np.max(wave)
            wave = torch.from_numpy(wave).float()
            channels.append(self.wave_transform(wave))
        
        return torch.cat(channels)
    
    def __getitem__(self, index):
        x = np.load(id_to_path(self.idxs[index], is_train=False)).astype(np.float32)
        image = self._frequency_transform(x)
        return image

test_dataset = GravWaveUnlabeledDataset(test_df['id'])
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size, num_workers=8)

CQT kernels created, time used = 0.0352 seconds


/opt/conda/lib/python3.7/site-packages/nnAudio/utils.py:326: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn('If fmax is given, n_bins will be ignored',SyntaxWarning)


In [17]:
model.eval()

predictions = np.zeros(len(test_dataset))
for i, images in tqdm(enumerate(test_dataloader), total=math.ceil(len(test_dataset) / config.batch_size)):
    predictions[i*config.batch_size: (i+1)*config.batch_size] = torch.sigmoid(model(images.to(device))).detach().cpu().numpy().squeeze()

  0%|          | 0/442 [00:00<?, ?it/s]

In [18]:
test_df['target'] = predictions
test_df.to_csv('submission.csv', index=False)

In [19]:
test_df

,id,target
0,00005bced6,0.998446
1,0000806717,0.985946
2,0000ef4fe1,0.539280
3,00020de251,0.937653
4,00024887b5,0.126428
...,...,...
225995,ffff4125f1,0.140387
225996,ffff9d32a6,0.395690
225997,ffff9f4c1f,0.194586
225998,ffffa19693,0.999891
